In [1]:
import json
from nltk import ngrams

In [2]:
def extract_headline_category():
    headline_category = []
    try:
        input_file = open('./News_Category_Dataset.json')
        input_data = input_file.readlines()
        input_file.close()
        for json_object in input_data:
            data = json.loads(json_object)
            headline_category.append((data['headline'], data['category']))
        return headline_category
    except IOError:
        print("ERROR : IO ERROR occurred while opening file")
        exit(0)


In [3]:
extract_headline_category()

[('There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV', 'CRIME'),
 ("Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song",
  'ENTERTAINMENT'),
 ('Hugh Grant Marries For The First Time At Age 57', 'ENTERTAINMENT'),
 ("Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork",
  'ENTERTAINMENT'),
 ('Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog',
  'ENTERTAINMENT'),
 ("Morgan Freeman 'Devastated' That Sexual Harassment Claims Could Undermine Legacy",
  'ENTERTAINMENT'),
 ("Donald Trump Is Lovin' New McDonald's Jingle In 'Tonight Show' Bit",
  'ENTERTAINMENT'),
 ('What To Watch On Amazon Prime That’s New This Week', 'ENTERTAINMENT'),
 ("Mike Myers Reveals He'd 'Like To' Do A Fourth Austin Powers Film",
  'ENTERTAINMENT'),
 ('What To Watch On Hulu That’s New This Week', 'ENTERTAINMENT'),
 ('Justin Timberlake Visits Texas School Shooting Victims', 'ENTERTAINMENT'),
 ("South Korean President Meets North Korea's Kim 

In [ ]:
def extract_n_grams(n, dataset:'list of tuple of headline'):
    headline_category_data = extract_headline_category()